In [114]:
import pandas as pd
import pygsheets
import holidays
from datetime import datetime, timedelta

In [7]:
# Authentifier avec le fichier de clé JSON
gc = pygsheets.authorize(service_file=r'D:\DATA\2024-11-2022_Tableau-Heures\feuille-heures-c0ab0678243d.json')

In [19]:
# Ouvrir la feuille Google Sheet par son titre
#spreadsheet = gc.open('2024-2026_TABLEAU DES HEURES')
spreadsheet = gc.open("RELEVE DES HEURES (REPONSES/NE PAS MODIFIER/FILTRER) ")

In [20]:
# Ouvrir la première feuille (DATA)
worksheet = spreadsheet.sheet1

In [71]:
#Importer toute les valeur et les convertir dans un df pandas
values = worksheet.get_all_values()
df = pd.DataFrame(values[1:], columns=values[0])


In [73]:
df = df.replace('', pd.NA)
df.tail(5)

,Timestamp,"Prénom, NOM",DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,...,Heures travaillées,Affaire,Score,Email Address,,,,,,
11504,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11505,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11506,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11507,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11508,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [46]:
df.head()

,Timestamp,"Prénom, NOM",DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,...,Heures travaillées,Affaire,Score,Email Address,,,,,,
0,2022-08-30,Jonathan EYRAUD,22/08/0022,ATELIER,NaT,5.00,ANTOFENOL - C2237199,Oui,4.00,LIFE-R-Oasys - LIFE20 ENV/FR/000186,...,NaT,NaT,NaT,jonathan.eyraud@chemdocwater.com,NaT,NaT,NaT,NaT,NaT,NaT
1,2024-10-25,Mehdi EL KANDOUSSI,10/18/0024,ATELIER,NaT,7,GRAND LAC - REUT - 24222172,Non,NaT,NaT,...,NaT,NaT,NaT,mehdi.el-kandoussi@chemdocwater.com,NaT,NaT,NaT,NaT,NaT,NaT
2,2024-03-05,François PIQUET,11/20/2004,BUREAU D'ETUDES,non,7,ISCHIA-KORDIANCE - 22272277,Non,NaT,NaT,...,NaT,NaT,NaT,piquetfrosty@gmail.com,NaT,NaT,NaT,NaT,NaT,NaT
3,2024-05-27,François PIQUET,11/20/2004,ATELIER,NaT,8,DS SMITH - 23201341,Non,NaT,NaT,...,NaT,NaT,NaT,piquetfrosty@gmail.com,NaT,NaT,NaT,NaT,NaT,NaT
4,2024-03-06,François PIQUET,11/20/2004,ATELIER,NaT,8,DS SMITH - 23201341,Non,NaT,NaT,...,NaT,NaT,NaT,piquetfrosty@gmail.com,NaT,NaT,NaT,NaT,NaT,NaT


In [74]:
#Supprimer lmes ligne sans valeur dans les colonnes Prénom Nom, Date
df = df.dropna(subset =["Prénom, NOM", "DATE"], how ="all").reset_index(drop =True)
print("Nombre de ligne conservées =",len(df))

Nombre de ligne conservées = 11250


In [ ]:
#PREFILTRAGE DES DATE : Convertis la colone timestamp pour reduire la taille du df car la colonne date est trop hétérogène
df["Timestamp"] = pd.to_datetime(df["Timestamp"],format='mixed')

#Créer une colonne anné a partir de timestamp puis nous réduison la taille du DF a partir des date supérieur ou égale 2023
df["Annee_timestamp"] = df["Timestamp"].dt.year
df = df[df["Annee_timestamp"] >= 2023]
print("Nombre de ligne conservées =",len(df))

In [76]:
#Modifie les années abhérante observée dans date 0024 et 2004
df["DATE"] = df["DATE"].apply(lambda x : x.replace("0024", "2024").replace("2004", "2024"))

In [ ]:
#Transforme la colonne date en datetime
df["DATE"] = pd.to_datetime(df["DATE"])

In [86]:
#Créé une colonne année, mois, n) de semaine a partir de la colonne DATE, les ligne 2022 sont peu donc conservées
df["Annee"] = df["DATE"].dt.year
df["Mois"] = df["DATE"].dt.month_name('fr_FR')
df["N_semaine"] = df["DATE"].dt.isocalendar().week
df.head(5)

,Timestamp,"Prénom, NOM",DATE,Lieu,Panier,Heures travaillées,Affaire,Autres déclarations,Heures travaillées,Affaire,...,,,,,,Annee_Date,Anne_timestamp,Annee,Mois,N_semaine
1,2024-10-25,Mehdi EL KANDOUSSI,2024-10-18,ATELIER,<NA>,7,GRAND LAC - REUT - 24222172,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,2024,Octobre,42
2,2024-03-05,François PIQUET,2024-11-20,BUREAU D'ETUDES,non,7,ISCHIA-KORDIANCE - 22272277,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,2024,Novembre,47
3,2024-05-27,François PIQUET,2024-11-20,ATELIER,<NA>,8,DS SMITH - 23201341,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,2024,Novembre,47
4,2024-03-06,François PIQUET,2024-11-20,ATELIER,<NA>,8,DS SMITH - 23201341,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,2024,2024,2024,Novembre,47
1243,2023-01-27,Lucien LIGOUT,2022-01-13,BUREAU D'ETUDES,non,7.00,SCP VALENSOLE - A2222487,Non,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,2022,2023,2022,Janvier,2


In [ ]:
#identification de la list des intérims
list_interim = ['TECHNICIEN MONTEUR1', 'ELECTROTECHNICIEN INDUSTRIEL2', 'MANUTENTIONNAIRE INDUSTRIEL1','ELECTROTECHNICIEN INDUSTRIEL1']

#inverser le prénom et le nom en conservant la liste interim dans le même sens 
def invert_name(name) :
    if name in list_interim :
        return name
    else : 
        invert = ' '.join(name.split()[1:] + [name.split()[0]])
        return invert

df["Prénom, NOM"] = df["Prénom, NOM"].apply(invert_name)
#Renomage de la colonne en NOM_PRENOM
df.rename(columns ={"Prénom, NOM":"NOM_Prénom"}, inplace = True)

In [ ]:
#Il y a plusieur colonne heure travaillé, nous renomons la première en HEURE_DECLAREE
df.columns.values[5] = "HEURE_DECLAREE"

In [107]:
#Transfomation de Heure déclaré en float
df["HEURE_DECLAREE"] = df["HEURE_DECLAREE"].astype(float)

In [ ]:
#Transformation de la colone panier en binaire 1 = oui pour pouvoir la compter
df["Panier"] = df["Panier"].fillna(0)
df["Panier"] = df["Panier"].apply(lambda x : 1 if x =="oui" else 0)


In [ ]:
#Compter les jours travailler si la valeur est différente de 1 = jours travaillé
df["JOUR_TRAVAILLEE"] = df["HEURE_DECLAREE"].apply(lambda x : 0 if x == 0 else 1)

In [125]:
#le DF est aggrégé par Année, Mois , et nuémro de semaine avec une somme par semaine sur heure déclarée et les paniers
df_agg = df.groupby(["Annee","Mois","N_semaine","NOM_Prénom"])[["HEURE_DECLAREE","Panier","JOUR_TRAVAILLEE"]].sum().reset_index()
#Supprésion de l'année 2022
df_agg = df_agg[df_agg["Annee"] > 2022]
#Triage par année et numéro de semaine 
df_agg = df_agg.sort_values(["Annee","NOM_Prénom","N_semaine"])
df_agg

,Annee,Mois,N_semaine,NOM_Prénom,HEURE_DECLAREE,Panier,JOUR_TRAVAILLEE
298,2023,Janvier,1,BALDINHO Francisco,35.0,0,5
310,2023,Janvier,2,BALDINHO Francisco,35.0,0,5
322,2023,Janvier,3,BALDINHO Francisco,35.0,0,5
333,2023,Janvier,4,BALDINHO Francisco,35.0,0,5
229,2023,Février,5,BALDINHO Francisco,21.0,0,3
...,...,...,...,...,...,...,...
1796,2024,Octobre,42,VIOLART Sébastien,35.5,0,4
1861,2024,Octobre,44,VIOLART Sébastien,35.5,0,4
1659,2024,Novembre,45,VIOLART Sébastien,32.0,0,4
1693,2024,Novembre,46,VIOLART Sébastien,24.0,0,3


In [126]:
# Obtenir les jours fériés en France pour les années du DataFrame
years = df_agg["Annee"].unique()
french_holidays = holidays.France(years=years)

# Fonction pour calculer les jours fériés par semaine
def get_holidays_per_week(year, week):
    start_date = pd.Timestamp(f"{year}-01-01") + pd.Timedelta(weeks=week - 1)
    end_date = start_date + pd.Timedelta(days=6)
    start_date = start_date.date()
    end_date = end_date.date()
    return sum(1 for date in french_holidays if start_date <= date <= end_date)

# Appliquer la fonction pour chaque ligne
df_agg["Jours_fériés"] = df_agg.apply(lambda row: get_holidays_per_week(row["Annee"], row["N_semaine"]), axis=1)

# Calculer le seuil d'heures travaillées en fonction des jours fériés
df_agg["Seuil_heures"] = 35 - df_agg["Jours_fériés"] * 7

# Calculer les heures supplémentaires
df_agg["Heures_supplementaires"] = df_agg["HEURE_DECLAREE"] - df_agg["Seuil_heures"]
df_agg["Heures_supplementaires"] = df_agg["Heures_supplementaires"].apply(lambda x: max(x, 0))  # Pas d'heures supp négatives

df_agg


,Annee,Mois,N_semaine,NOM_Prénom,HEURE_DECLAREE,Panier,JOUR_TRAVAILLEE,Jours_fériés,Seuil_heures,Heures_supplementaires
298,2023,Janvier,1,BALDINHO Francisco,35.0,0,5,1,28,7.0
310,2023,Janvier,2,BALDINHO Francisco,35.0,0,5,0,35,0.0
322,2023,Janvier,3,BALDINHO Francisco,35.0,0,5,0,35,0.0
333,2023,Janvier,4,BALDINHO Francisco,35.0,0,5,0,35,0.0
229,2023,Février,5,BALDINHO Francisco,21.0,0,3,0,35,0.0
...,...,...,...,...,...,...,...,...,...,...
1796,2024,Octobre,42,VIOLART Sébastien,35.5,0,4,0,35,0.5
1861,2024,Octobre,44,VIOLART Sébastien,35.5,0,4,1,28,7.5
1659,2024,Novembre,45,VIOLART Sébastien,32.0,0,4,0,35,0.0
1693,2024,Novembre,46,VIOLART Sébastien,24.0,0,3,1,28,0.0


In [128]:
#Convertis les point en virgule pour google sheet
df_agg = df_agg.applymap( lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, float) else x)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_54820\3436681036.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_agg = df_agg.applymap( lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, float) else x)


In [ ]:
#exportation des données dans google sheet
spreadsheet_export = gc.open('2024-2026_TABLEAU DES HEURES')
#Choisis lapremière feuille
worksheet = spreadsheet_export[0]
#selectionnne la cellule d'arrivé

worksheet.set_dataframe(df_agg, (1, 1))